In [2]:
from frontend.plot_maps import TripMap
from utils.constants import StationIds
import folium
from streamlit_folium import st_folium

def home():
    st.markdown("# Startsida")
def reseplanerare():
    st.markdown("# 🗺️ Reseplanerare")
    st.markdown(
        """
        Den här dashboarden syftar till att både utforska data för olika platser, 
        men ska även fungera som en reseplanerare där du får välja och planera din resa.
        """
    )

    # Skapa en Folium-karta
    m = folium.Map(location=[63.8258, 20.2630], zoom_start=6)  # Umeå koordinater
    folium.Marker([55.6050, 13.0038], tooltip="Malmö").add_to(m)  # Malmö koordinater

    # Visa kartan i Streamlit
    st_folium(m, width=700, height=500)

    # Eventuell framtida logik för origin_id och destination_id
    origin_id = "740000003"  
    destination_id = "740000190" 
    st.markdown(f"### Ursprung: {origin_id}")
    st.markdown(f"### Destination: {destination_id}")
    

2025-01-27 16:07:09.830 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [44]:
from backend.trips import TripPlanner
from backend.connect_to_api import ResRobot
import pandas as pd
import requests
import folium

resrobot = ResRobot()
origin_id = input("Enter your location: ")
dest_id = input("Enter your destination: ")    

In [45]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime

# Load API key from .env file
load_dotenv()
API_KEY = os.getenv("API_KEY")


class DeparturePlanner:
    """
    A planner that retrieves departure information between two locations
    using the Resrobot API.
    """

    def __init__(self, origin_location: str, destination_location: str):
        """
        :param origin_location: The textual origin (e.g., 'Stockholm')
        :param destination_location: The textual destination (e.g., 'Göteborg')
        """
        self.origin_location = origin_location
        self.destination_location = destination_location

        # Convert textual locations to stop IDs
        self.origin_id = self.access_id_from_location(self.origin_location)
        self.dest_id = self.access_id_from_location(self.destination_location)

        # Validate stop IDs
        if not self.origin_id or not self.dest_id:
            raise ValueError(f"Invalid stop IDs: {self.origin_id}, {self.dest_id}")

        # Fetch trips between origin and destination
        self.trips = self.fetch_trips()
        if not self.trips:
            print(f"No trips found between {self.origin_location} and {self.destination_location}.")
            self.trips = []

    def access_id_from_location(self, location: str) -> str | None:
        """
        Use the Resrobot location.name endpoint to find a location's stop ID.
        :param location: A textual name (e.g., 'Stockholm')
        :return: The first ID (as a string) if found, else None.
        """
        url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            result = response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request to Resrobot failed for location '{location}': {e}")
            return None

        stop_locations = result.get("StopLocation")
        if stop_locations and isinstance(stop_locations, list):
            return stop_locations[0].get("id")
        return None

    def fetch_trips(self) -> list | None:
        """
        Fetch trips between origin and destination using Resrobot's trip endpoint.
        :return: List of trips as dictionaries or None if the request fails.
        """
        url = (
            f"https://api.resrobot.se/v2.1/trip?format=json"
            f"&originId={self.origin_id}&destId={self.dest_id}"
            f"&passlist=true&showPassingPoints=true&accessId={API_KEY}"
        )
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json().get("Trip")
        except requests.exceptions.RequestException as e:
            print(f"Request to Resrobot failed: {e}")
            return None

    def next_available_trips_today(self) -> list[pd.DataFrame]:
        """
        Fetch today's available trips between origin and destination.
        Each trip is represented as a DataFrame of stops (including arrival and departure times).
        :return: A list of DataFrames, each corresponding to one trip.
        """
        if not self.trips:
            return []

        today_str = datetime.now().date().strftime("%Y-%m-%d")
        trips_today = []

        for trip in self.trips:
            if not isinstance(trip, dict):
                continue

            leglist = trip.get("LegList", {}).get("Leg", [])
            if not leglist:
                continue

            # Create a DataFrame for the legs
            df_legs = pd.DataFrame(leglist)

            # Normalize 'Stops' if present
            if "Stops" in df_legs.columns:
                df_stops = pd.json_normalize(
                    df_legs["Stops"].dropna(), 
                    "Stop", 
                    errors="ignore"
                )
                df_stops["time"] = df_stops["arrTime"].fillna(df_stops["depTime"])
                df_stops["date"] = df_stops["arrDate"].fillna(df_stops["depDate"])

                # If the trip is today, add it to the result
                if not df_stops.empty and df_stops["date"].iloc[0] == today_str:
                    trips_today.append(
                        df_stops[
                            [
                                "name", 
                                "extId", 
                                "lon", 
                                "lat", 
                                "depTime", 
                                "depDate", 
                                "arrTime", 
                                "arrDate", 
                                "time", 
                                "date"
                            ]
                        ]
                    )
        return trips_today

    def debug_trips(self):
        """
        Debugging method to log API response.
        """
        print(f"Debugging trips between {self.origin_id} and {self.dest_id}...")
        print(self.trips)


# Example usage
if __name__ == "__main__":
    try:
        planner = DeparturePlanner("Stockholm", "Göteborg")
        trips_today = planner.next_available_trips_today()

        if trips_today:
            print("Here are the trips available today:")
            for idx, trip_df in enumerate(trips_today, start=1):
                print(f"\nTrip {idx}:\n{trip_df}\n")
        else:
            print("No trips available today.")
    except Exception as e:
        print(f"An error occurred: {e}")


Request to Resrobot failed for location 'Stockholm': 401 Client Error: Unauthorized for url: https://api.resrobot.se/v2.1/location.name?input=Stockholm&format=json&accessId=a91ab4c63147441e82540a8a02c3cb52
Request to Resrobot failed for location 'Göteborg': 401 Client Error: Unauthorized for url: https://api.resrobot.se/v2.1/location.name?input=G%C3%B6teborg&format=json&accessId=a91ab4c63147441e82540a8a02c3cb52
An error occurred: Invalid stop IDs: None, None


In [46]:
import requests

API_KEY = "DIN_NYA_API_NYCKEL"

# Testar location.name
location_url = f"https://api.resrobot.se/v2.1/location.name?input=Stockholm&format=json&accessId={API_KEY}"
response = requests.get(location_url)
if response.status_code == 200:
    print("Location.name call successful!")
    print(response.json())
else:
    print(f"Location.name call failed: {response.status_code}, {response.text}")

# Testar trip
trip_url = (
    f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000001&destId=740000002"
    f"&passlist=true&showPassingPoints=true&accessId={API_KEY}"
)
response = requests.get(trip_url)
if response.status_code == 200:
    print("Trip call successful!")
    print(response.json())
else:
    print(f"Trip call failed: {response.status_code}, {response.text}")


Location.name call failed: 401, {"serverVersion":"2.45.1","dialectVersion":"2.45","errorCode":"API_AUTH","errorText":"access denied for DIN_NYA_API_NYCKEL on location.name identified by location.name","requestId":"default-request-id"}
Trip call failed: 401, {"serverVersion":"2.45.1","dialectVersion":"2.45","errorCode":"API_AUTH","errorText":"access denied for DIN_NYA_API_NYCKEL on tripSearch identified by tripSearch","requestId":"default-request-id"}


In [47]:
print("Response is None")


Response is None


In [48]:
{"errorCode": 1001, "errorText": "Invalid access ID"}


{'errorCode': 1001, 'errorText': 'Invalid access ID'}

In [49]:
import requests

API_KEY = "din_giltiga_api_nyckel"
test_url = f"https://api.resrobot.se/v2.1/location.name?input=Stockholm&format=json&accessId={API_KEY}"
response = requests.get(test_url)
print(response.json())


{'serverVersion': '2.45.1', 'dialectVersion': '2.45', 'errorCode': 'API_AUTH', 'errorText': 'access denied for din_giltiga_api_nyckel on location.name identified by location.name', 'requestId': 'default-request-id'}


In [50]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY = os.getenv("API_KEY")
if not API_KEY:
    print("API_KEY not found. Check your .env file.")
else:
    print(f"Using API_KEY: {API_KEY}")

    # Testa ett enkelt API-anrop
    test_url = f"https://api.resrobot.se/v2.1/location.name?input=Stockholm&format=json&accessId={API_KEY}"
    response = requests.get(test_url)

    if response.status_code == 200:
        print("API call successful. Response:")
        print(response.json())
    else:
        print(f"API call failed. Status code: {response.status_code}")
        print(response.text)


Using API_KEY: a91ab4c63147441e82540a8a02c3cb52
API call failed. Status code: 401
{"serverVersion":"2.45.1","dialectVersion":"2.45","errorCode":"API_AUTH","errorText":"access denied for a91ab4c63147441e82540a8a02c3cb52 on location.name identified by location.name","requestId":"default-request-id"}


In [51]:
def access_id_from_location(self, location: str) -> str | None:
    url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch location for {location}. Status code: {response.status_code}")
        return None

    result = response.json()
    print(f"Location response for {location}: {result}")  # Debug-utskrift

    stop_locations = result.get("StopLocation")
    if stop_locations and isinstance(stop_locations, list):
        return stop_locations[0].get("id")
    return None
